In [108]:
import pandas as pd
import numpy as np
import re

In [109]:
df_ps = pd.read_csv('./data/googleplaystore.csv')
pd.set_option('display.max_row', 20)
pd.set_option('display.max_columns', 500)

df_ps.head(10)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [110]:
#delete NAN
df_ps = df_ps.dropna()

unique = df_ps['App'].unique()

len(unique)

8190

In [111]:
df_ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null object
Size              9360 non-null object
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null object
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null object
Android Ver       9360 non-null object
dtypes: float64(1), object(12)
memory usage: 585.0+ KB


In [112]:
df_ps_review = pd.read_csv('./data/googleplaystore_user_reviews.csv')
df_ps_review.head(10)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000
6,10 Best Foods for You,Amazing,Positive,0.60,0.900000
7,10 Best Foods for You,NaN,NaN,NaN,NaN
8,10 Best Foods for You,"Looking forward app,",Neutral,0.00,0.000000
9,10 Best Foods for You,It helpful site ! It help foods get !,Neutral,0.00,0.000000


In [113]:
df_ps_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
App                       64295 non-null object
Translated_Review         37427 non-null object
Sentiment                 37432 non-null object
Sentiment_Polarity        37432 non-null float64
Sentiment_Subjectivity    37432 non-null float64
dtypes: float64(2), object(3)
memory usage: 1.7+ MB


In [114]:
#The row 10472 is problematic, it doesn't have the Category camp so everything is shifted to the left
#df_ps.loc[[10472]]

## A. All groups and individual must do the following

#### 1. Convert the app sizes to a number

In [115]:
#Checking the format of the data
df_ps.Size.unique()

array(['19M', '14M', '8.7M', '25M', '2.8M', '5.6M', '29M', '33M', '3.1M',
       '28M', '12M', '20M', '21M', '37M', '5.5M', '17M', '39M', '31M',
       '4.2M', '23M', '6.0M', '6.1M', '4.6M', '9.2M', '5.2M', '11M',
       '24M', 'Varies with device', '9.4M', '15M', '10M', '1.2M', '26M',
       '8.0M', '7.9M', '56M', '57M', '35M', '54M', '201k', '3.6M', '5.7M',
       '8.6M', '2.4M', '27M', '2.7M', '2.5M', '7.0M', '16M', '3.4M',
       '8.9M', '3.9M', '2.9M', '38M', '32M', '5.4M', '18M', '1.1M',
       '2.2M', '4.5M', '9.8M', '52M', '9.0M', '6.7M', '30M', '2.6M',
       '7.1M', '22M', '6.4M', '3.2M', '8.2M', '4.9M', '9.5M', '5.0M',
       '5.9M', '13M', '73M', '6.8M', '3.5M', '4.0M', '2.3M', '2.1M',
       '42M', '9.1M', '55M', '23k', '7.3M', '6.5M', '1.5M', '7.5M', '51M',
       '41M', '48M', '8.5M', '46M', '8.3M', '4.3M', '4.7M', '3.3M', '40M',
       '7.8M', '8.8M', '6.6M', '5.1M', '61M', '66M', '79k', '8.4M',
       '3.7M', '118k', '44M', '695k', '1.6M', '6.2M', '53M', '1.4M',
      

In [116]:
# create new column Size (Megabite) from Size
#df_ps['Size (Megabite)'] = df_ps['Size']

# understand Size column composition
k_bool = df_ps['Size'].str.extract(r"(\d*\S?\d*)[k]").notnull()[0]
k_values = df_ps['Size'].loc[k_bool != False]
print(k_values)

58       201k
209       23k
384       79k
450      118k
458      695k
768      556k
775      526k
1019     334k
1521     232k
1523     624k
         ... 
10675     89k
10677    234k
10678    257k
10688    364k
10690    861k
10732    467k
10755    676k
10763    552k
10832    582k
10833    619k
Name: Size, Length: 257, dtype: object


In [117]:
# understand Size column composition
m_bool = df_ps['Size'].str.extract(r"(\d*\S?\d*)[M]").notnull()[0]
m_values = df_ps['Size'].loc[m_bool != False]
print(m_values)

0         19M
1         14M
2        8.7M
3         25M
4        2.8M
5        5.6M
6         19M
7         29M
8         33M
9        3.1M
         ... 
10819    3.6M
10820    8.6M
10827     13M
10828     13M
10829    7.4M
10830    2.3M
10834    2.6M
10836     53M
10837    3.6M
10840     19M
Name: Size, Length: 7466, dtype: object


In [118]:
size_to_num = re.compile('(?P<number>\d+\.{0,1}\d*)(?P<prefix>\w*)')

#In this function we use lowercase k for kilo and uppercase M and G for Mega and Giga. 
#We Assume no file with any other unit otherwise the unit will be ignored.
def prefix_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000
    if unit == 'k':
        return 1000
    return 1

#This function can understand the format of the data given to it using regex,
#If the given data is not in the expected format it will return 'Not a Number'
#For example 'Varies with device' is transformed in NaN
def ConvertSizeToByte(Size):
    searched = size_to_num.search(Size)
    if searched is None:
        return np.nan
    else:
        prefix = searched.group('prefix')
        mult = prefix_to_mult(prefix)
        result = float(searched.group('number'))
        return int(result*mult)

In [119]:
# chose to create a new column for the converted data
df_ps['SizeInBytes'] = df_ps['Size'].apply(ConvertSizeToByte)
df_ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0


In [120]:
#Use tail because it shows a NaN
df_ps[['App','Size','SizeInBytes']].tail()

,App,Size,SizeInBytes
10834,FR Calculator,2.6M,2600000.0
10836,Sya9a Maroc - FR,53M,53000000.0
10837,Fr. Mike Schmitz Audio Teachings,3.6M,3600000.0
10839,The SCP Foundation DB fr nn5n,Varies with device,NaN
10840,iHoroscope - 2018 Daily Horoscope & Astrology,19M,19000000.0


In [121]:
df_ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 14 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null object
Size              9360 non-null object
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null object
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null object
Android Ver       9360 non-null object
SizeInBytes       7723 non-null float64
dtypes: float64(2), object(12)
memory usage: 658.1+ KB


#### 2. Convert the number of installs to a number

In [122]:
#Checking the format of the data
df_ps.Installs.unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '100+', '500+', '10+',
       '5+', '50+', '1+'], dtype=object)

In [123]:
#checking if there are other letters in Installs
df_ps["Installs"].str.extractall(r"([A-z])", flags = re.IGNORECASE ).count()

0    0
dtype: int64

In [124]:
df_ps['Installs'].str.extractall(r"([A-Za-z])")

,,0
,match,


In [125]:
df_ps["Installs"].str.extractall(r"([^0-9\,])")

,,0
,match,
0,0,+
1,0,+
2,0,+
3,0,+
4,0,+
5,0,+
6,0,+
7,0,+
8,0,+


In [126]:
# removing '+' sign and ','
install_to_num = re.compile('(?P<number>[\d,]+)') #the format is num,num,num,...,num

#We find using regex the number in the format 
def installToNumber(installs):
    found = install_to_num.search(installs)
    if(found):
        replacedComma = found.group('number').replace(',','') #replace the commas with nothing
        return int(replacedComma) 
    else:
        return np.nan
    

In [127]:
df_ps['InstallNumber']=df_ps['Installs'].apply(installToNumber)

df_ps[['App','Installs','InstallNumber']].head()

,App,Installs,InstallNumber
0,Photo Editor & Candy Camera & Grid & ScrapBook,"10,000+",10000
1,Coloring book moana,"500,000+",500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...","5,000,000+",5000000
3,Sketch - Draw & Paint,"50,000,000+",50000000
4,Pixel Draw - Number Art Coloring Book,"100,000+",100000


#### 3. Transform “Varies with device” into a missing value

In [128]:
#replace 'Varies with device' with NaN.
df_ps.replace('Varies with device', np.nan,inplace=True)
#inplace is needed so the replace is executed on the dataframe itself

In [129]:
df_ps[df_ps == "Varies with device"].count() #checkig if there is others "Varies with device" inside

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
SizeInBytes       0
InstallNumber     0
dtype: int64

#### 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [130]:
#The versions with the W in the end are the wearable version we will not consider the W
df_ps['Android Ver'].value_counts()

4.1 and up       2059
4.0.3 and up     1240
4.0 and up       1129
4.4 and up        875
2.3 and up        582
5.0 and up        535
4.2 and up        337
2.3.3 and up      240
3.0 and up        211
2.2 and up        208
                 ... 
2.0.1 and up        7
4.4W and up         6
8.0 and up          5
7.1 and up          3
1.0 and up          2
5.0 - 8.0           2
4.0.3 - 7.1.1       2
5.0 - 6.0           1
4.1 - 7.1.1         1
7.0 - 7.1.1         1
Name: Android Ver, Length: 30, dtype: int64

In [131]:
# deleting any letter from Android Ver and Current Ver in order to leave only the numerical part
df_ps["Android Ver"].replace("[a-z]+|[a-z]+", value = "", regex = True, inplace = True)
df_ps["Current Ver"].replace("[a-z]+|[a-z]+", value = "", regex = True, inplace = True)
# deleting also the last '.' if any
df_ps["Current Ver"].replace(r"(\.+$)", value = "", regex = True, inplace = True)

In [132]:
df_ps['Current Ver'].head(30)

0           1.0.0
1           2.0.0
2           1.2.4
3             NaN
4             1.1
5             1.0
6             1.1
7        6.1.61.1
8           2.9.2
9             2.8
         ...     
21            1.1
22        2.2.6.2
24          1.1.3
25            1.5
26          1.0.8
27           1.03
28            6.0
29            1.0
30    6.7.12.2018
31            1.2
Name: Current Ver, Length: 30, dtype: object

In [133]:
df_ps['Android Ver'].head(30)

0     4.0.3  
1     4.0.3  
2     4.0.3  
3       4.2  
4       4.4  
5       2.3  
6     4.0.3  
7       4.2  
8       3.0  
9     4.0.3  
       ...   
21    4.0.3  
22    4.0.3  
24      4.1  
25      3.0  
26    4.0.3  
27    4.0.3  
28      2.3  
29      2.3  
30    4.0.3  
31      2.3  
Name: Android Ver, Length: 30, dtype: object

#### 5. Remove the duplicates

In [134]:
# count row before drop duplicates
df_ps.shape

(9360, 15)

In [135]:
# checking duplicated app
df_ps[df_ps.duplicated('App')].shape[0]

1170

In [136]:
df_ps.Reviews

0           159
1           967
2         87510
3        215644
4           967
5           167
6           178
7         36815
8         13791
9           121
          ...  
10828       291
10829       603
10830       881
10832      1195
10833        44
10834         7
10836        38
10837         4
10839       114
10840    398307
Name: Reviews, Length: 9360, dtype: object

In [137]:
df_ps.Reviews = df_ps.Reviews.apply(pd.to_numeric)

In [138]:
df_ps.Reviews.dtype

dtype('int64')

In [139]:
# dropping duplicates rows
df_ps=df_ps.sort_values(['App','Reviews'],ascending=False).drop_duplicates('App',keep='first')
df_ps

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0,Everyone,Entertainment,"July 14, 2018",1.1.3.2,4.0.3,4000000.0,1000000
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0,Everyone,Tools,"October 21, 2017",1.0,4.2,3200000.0,10000
2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0,Everyone,Social,"July 24, 2018",4.2.4,4.0.3,18000000.0,1000000
4362,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,$399.99,Everyone,Lifestyle,"March 11, 2018",1.0.0,4.4,26000000.0,10000
6420,감성학원 BL 첫사랑,COMICS,4.4,190,34M,"10,000+",Free,0,Everyone,Comics,"November 17, 2017",3.2.1,3.0,34000000.0,10000
9222,英漢字典 EC Dictionary,FAMILY,4.3,55408,NaN,"1,000,000+",Free,0,Everyone,Education,"January 8, 2018",NaN,NaN,NaN,1000000
313,"漫咖 Comics - Manga,Novel and Stories",COMICS,4.1,12088,21M,"1,000,000+",Free,0,Mature 17+,Comics,"July 6, 2018",2.3.1,4.0.3,21000000.0,1000000
5698,日本AV历史,BOOKS_AND_REFERENCE,4.1,215,30M,"10,000+",Free,0,Teen,Books & Reference,"March 6, 2018",1.2,4.0,30000000.0,10000
3824,乗換NAVITIME Timetable & Route Search in Japan T...,MAPS_AND_NAVIGATION,4.4,50459,NaN,"5,000,000+",Free,0,Everyone,Maps & Navigation,"July 26, 2018",NaN,NaN,NaN,5000000
1488,"乐屋网: Buying a house, selling a house, renting ...",HOUSE_AND_HOME,3.7,2248,15M,"100,000+",Free,0,Everyone,House & Home,"August 3, 2018",3.1.1,4.0,15000000.0,100000


In [140]:
#There are no more duplicates
df_ps[df_ps.duplicated('App')].shape[0]

0

In [141]:
df_ps[df_ps["App"] == "Coloring book moana"] #check the indexes of this App

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber
2033,Coloring book moana,FAMILY,3.9,974,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3,14000000.0,500000


#### 6. For each category, compute the number of apps

In [142]:
df_ps.groupby(df_ps['Category'], as_index = False)['App'].count()

,Category,App
0,ART_AND_DESIGN,58
1,AUTO_AND_VEHICLES,73
2,BEAUTY,42
3,BOOKS_AND_REFERENCE,169
4,BUSINESS,263
5,COMICS,54
6,COMMUNICATION,256
7,DATING,133
8,EDUCATION,106
9,ENTERTAINMENT,87


#### 7. For each category, compute the average rating

In [143]:
df_ps.groupby(df_ps['Category'], as_index = False)['Rating'].mean()

,Category,Rating
0,ART_AND_DESIGN,4.379310
1,AUTO_AND_VEHICLES,4.190411
2,BEAUTY,4.278571
3,BOOKS_AND_REFERENCE,4.344970
4,BUSINESS,4.098479
5,COMICS,4.181481
6,COMMUNICATION,4.121484
7,DATING,3.980451
8,EDUCATION,4.354717
9,ENTERTAINMENT,4.129885


#### 8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [144]:
# I create a new dataframe with only the columns of interest
df_genres = df_ps[["Genres"]]
#checking the value count
df_genres['Genres'].value_counts()

Tools                                 718
Entertainment                         471
Education                             429
Finance                               302
Productivity                          301
Lifestyle                             300
Personalization                       296
Action                                292
Medical                               290
Sports                                266
                                     ... 
Lifestyle;Pretend Play                  1
Communication;Creativity                1
Video Players & Editors;Creativity      1
Adventure;Education                     1
Comics;Creativity                       1
Role Playing;Brain Games                1
Travel & Local;Action & Adventure       1
Adventure;Brain Games                   1
Puzzle;Education                        1
Health & Fitness;Education              1
Name: Genres, Length: 114, dtype: int64

In [145]:
#splits the genres after the ";" to have a series divided into multindexes: 0 and 1
gens = df_genres['Genres'].str.split(";", expand = True)
df_genres['Genres_1'] = gens[0]
df_genres['Genres_2'] = gens[1]
df_genres.drop(['Genres'], axis=1, inplace = True)


c:\users\danie\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\danie\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\users\danie\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value i

In [146]:
df_genres['Genres_2'].value_counts()

Action & Adventure    96
Education             89
Pretend Play          68
Brain Games           62
Creativity            32
Music & Video         31
Name: Genres_2, dtype: int64

In [147]:
# create the App dataframe
df_app = df_ps[['App']]
df_app.head()

,App
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!
2575,"💘 WhatsLov: Smileys of love, stickers and GIF"
4362,💎 I'm rich
6420,감성학원 BL 첫사랑


In [148]:
df_genres.head()

,Genres_1,Genres_2
882,Entertainment,None
7559,Tools,None
2575,Social,None
4362,Lifestyle,None
6420,Comics,None


In [149]:
# gettin one column for genres
# merge apps with genres and then melt them by app column
# melt is useful for having only one column for genres 
df_app_gen = pd.merge(df_app, df_genres, left_index = True, right_index = True)\
               .melt(id_vars = ['App'], value_name = "Genres").drop(columns = "variable") 
df_app_gen

,App,Genres
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,Entertainment
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,Tools
2,"💘 WhatsLov: Smileys of love, stickers and GIF",Social
3,💎 I'm rich,Lifestyle
4,감성학원 BL 첫사랑,Comics
5,英漢字典 EC Dictionary,Education
6,"漫咖 Comics - Manga,Novel and Stories",Comics
7,日本AV历史,Books & Reference
8,乗換NAVITIME Timetable & Route Search in Japan T...,Maps & Navigation
9,"乐屋网: Buying a house, selling a house, renting ...",House & Home


In [150]:
# drop N/A values
df_app_gen = df_app_gen.dropna() #delete the remaining NAN
df_app_gen

,App,Genres
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,Entertainment
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,Tools
2,"💘 WhatsLov: Smileys of love, stickers and GIF",Social
3,💎 I'm rich,Lifestyle
4,감성학원 BL 첫사랑,Comics
5,英漢字典 EC Dictionary,Education
6,"漫咖 Comics - Manga,Novel and Stories",Comics
7,日本AV历史,Books & Reference
8,乗換NAVITIME Timetable & Route Search in Japan T...,Maps & Navigation
9,"乐屋网: Buying a house, selling a house, renting ...",House & Home


In [151]:
df_app_gen.info() #check composition of the dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8568 entries, 0 to 16254
Data columns (total 2 columns):
App       8568 non-null object
Genres    8568 non-null object
dtypes: object(2)
memory usage: 133.9+ KB


In [152]:
# check if everything work fine
df_app_gen.loc[df_app_gen['App'] == "Pixel Draw - Number Art Coloring Book"]

,App,Genres
2329,Pixel Draw - Number Art Coloring Book,Art & Design
10519,Pixel Draw - Number Art Coloring Book,Creativity


#### 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [153]:
# concate of orignal df and the genres1 genres2 binaryzed columns
df_ps_2 = pd.concat([df_ps, pd.get_dummies(df_genres["Genres_1"], dtype = "bool")], axis=1).drop(['Genres'], axis=1)
df_ps_2 = pd.concat([df_ps_2, pd.get_dummies(df_genres["Genres_2"], dtype = "bool")], axis=1)

df_ps_2.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Last Updated', 'Current Ver', 'Android Ver',
       'SizeInBytes', 'InstallNumber', 'Action', 'Adventure', 'Arcade',
       'Art & Design', 'Auto & Vehicles', 'Beauty', 'Board',
       'Books & Reference', 'Business', 'Card', 'Casino', 'Casual', 'Comics',
       'Communication', 'Dating', 'Education', 'Educational', 'Entertainment',
       'Events', 'Finance', 'Food & Drink', 'Health & Fitness', 'House & Home',
       'Libraries & Demo', 'Lifestyle', 'Maps & Navigation', 'Medical',
       'Music', 'Music & Audio', 'News & Magazines', 'Parenting',
       'Personalization', 'Photography', 'Productivity', 'Puzzle', 'Racing',
       'Role Playing', 'Shopping', 'Simulation', 'Social', 'Sports',
       'Strategy', 'Tools', 'Travel & Local', 'Trivia',
       'Video Players & Editors', 'Weather', 'Word', 'Action & Adventure',
       'Brain Games', 'Creativity', 'Education', 'Music & Vi

In [154]:
# checking if everthing works fine
df_ps_2[["App", "Art & Design", "Creativity"]].loc[df_ps_2['App'] == "Pixel Draw - Number Art Coloring Book"]

,App,Art & Design,Creativity
4,Pixel Draw - Number Art Coloring Book,True,True


#### 10. For each genre, compute the average rating. What is the genre with highest average?

In [155]:
#joining
df_ps_3 = df_app_gen.merge(df_ps, on = "App")
df_ps_3.drop(['Genres_y'], axis=1, inplace = True) # dropping unuseful columns
df_ps_3.head()

,App,Genres_x,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber
0,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,Entertainment,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0,Everyone,"July 14, 2018",1.1.3.2,4.0.3,4000000.0,1000000
1,📏 Smart Ruler ↔️ cm/inch measuring for homework!,Tools,TOOLS,4.0,19,3.2M,"10,000+",Free,0,Everyone,"October 21, 2017",1.0,4.2,3200000.0,10000
2,"💘 WhatsLov: Smileys of love, stickers and GIF",Social,SOCIAL,4.6,22098,18M,"1,000,000+",Free,0,Everyone,"July 24, 2018",4.2.4,4.0.3,18000000.0,1000000
3,💎 I'm rich,Lifestyle,LIFESTYLE,3.8,718,26M,"10,000+",Paid,$399.99,Everyone,"March 11, 2018",1.0.0,4.4,26000000.0,10000
4,감성학원 BL 첫사랑,Comics,COMICS,4.4,190,34M,"10,000+",Free,0,Everyone,"November 17, 2017",3.2.1,3.0,34000000.0,10000


In [156]:
df_ps_3.groupby('Genres_x', as_index=False, sort=False)[['Rating']].mean().head(10)

,Genres_x,Rating
0,Entertainment,4.093028
1,Tools,4.040056
2,Social,4.247291
3,Lifestyle,4.094040
4,Comics,4.181481
5,Education,4.290290
6,Books & Reference,4.343275
7,Maps & Navigation,4.036441
8,House & Home,4.140984
9,Simulation,4.154500


#### 11. For each app, compute the approximate income, obtain as a product of number of installs and price.

In [157]:
#len(df_ps['App'].unique())

In [158]:
df_ps["Price"].unique()

array(['0', '$399.99', '$2.99', '$1.99', '$1.20', '$0.99', '$4.29',
       '$4.99', '$4.49', '$3.99', '$10.00', '$5.49', '$19.40', '$5.99',
       '$12.99', '$2.50', '$2.56', '$9.99', '$1.49', '$2.90', '$1.00',
       '$2.49', '$79.99', '$19.99', '$16.99', '$1.97', '$3.49', '$17.99',
       '$6.99', '$2.95', '$4.59', '$7.49', '$10.99', '$4.84', '$1.76',
       '$7.99', '$29.99', '$4.60', '$3.02', '$1.70', '$15.99', '$24.99',
       '$9.00', '$1.59', '$8.99', '$1.61', '$39.99', '$15.46', '$400.00',
       '$299.99', '$379.99', '$18.99', '$37.99', '$389.99', '$8.49',
       '$14.99', '$3.88', '$13.99', '$11.99', '$4.77', '$3.28', '$3.95',
       '$14.00', '$2.59', '$2.00', '$1.75', '$3.90', '$1.50', '$1.29',
       '$6.49', '$3.04', '$33.99', '$3.08'], dtype=object)

In [159]:
# removing dollar $
df_ps["Price"].replace("\$+", value = "", regex = True,inplace = True)
# checkin if there are any char that is not a number or '.''
df_ps["Price"].str.extractall(r"([^0-9\.])")

,,0
,match,


In [160]:
df_ps["Price"].replace("Everyone", value = "", regex = True, inplace = True)

#casting to float
df_ps['Price'] = df_ps['Price'].astype(np.float)

df_ps.head(15)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0.00,Everyone,Entertainment,"July 14, 2018",1.1.3.2,4.0.3,4000000.0,1000000
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0.00,Everyone,Tools,"October 21, 2017",1.0,4.2,3200000.0,10000
2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0.00,Everyone,Social,"July 24, 2018",4.2.4,4.0.3,18000000.0,1000000
4362,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,399.99,Everyone,Lifestyle,"March 11, 2018",1.0.0,4.4,26000000.0,10000
6420,감성학원 BL 첫사랑,COMICS,4.4,190,34M,"10,000+",Free,0.00,Everyone,Comics,"November 17, 2017",3.2.1,3.0,34000000.0,10000
9222,英漢字典 EC Dictionary,FAMILY,4.3,55408,NaN,"1,000,000+",Free,0.00,Everyone,Education,"January 8, 2018",NaN,NaN,NaN,1000000
313,"漫咖 Comics - Manga,Novel and Stories",COMICS,4.1,12088,21M,"1,000,000+",Free,0.00,Mature 17+,Comics,"July 6, 2018",2.3.1,4.0.3,21000000.0,1000000
5698,日本AV历史,BOOKS_AND_REFERENCE,4.1,215,30M,"10,000+",Free,0.00,Teen,Books & Reference,"March 6, 2018",1.2,4.0,30000000.0,10000
3824,乗換NAVITIME Timetable & Route Search in Japan T...,MAPS_AND_NAVIGATION,4.4,50459,NaN,"5,000,000+",Free,0.00,Everyone,Maps & Navigation,"July 26, 2018",NaN,NaN,NaN,5000000
1488,"乐屋网: Buying a house, selling a house, renting ...",HOUSE_AND_HOME,3.7,2248,15M,"100,000+",Free,0.00,Everyone,House & Home,"August 3, 2018",3.1.1,4.0,15000000.0,100000


In [162]:
df_ps['Income'] = df_ps['Price'] * df_ps['InstallNumber']
df_ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Income
882,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,ENTERTAINMENT,4.7,11661,4.0M,"1,000,000+",Free,0.00,Everyone,Entertainment,"July 14, 2018",1.1.3.2,4.0.3,4000000.0,1000000,0.0
7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,3.2M,"10,000+",Free,0.00,Everyone,Tools,"October 21, 2017",1.0,4.2,3200000.0,10000,0.0
2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18M,"1,000,000+",Free,0.00,Everyone,Social,"July 24, 2018",4.2.4,4.0.3,18000000.0,1000000,0.0
4362,💎 I'm rich,LIFESTYLE,3.8,718,26M,"10,000+",Paid,399.99,Everyone,Lifestyle,"March 11, 2018",1.0.0,4.4,26000000.0,10000,3999900.0
6420,감성학원 BL 첫사랑,COMICS,4.4,190,34M,"10,000+",Free,0.00,Everyone,Comics,"November 17, 2017",3.2.1,3.0,34000000.0,10000,0.0


In [163]:
df_ps.groupby('App', as_index=False, sort=True)[['Income']].sum()

,App,Income
0,+Download 4 Instagram Twitter,0.0
1,- Free Comics - Comic Apps,0.0
2,.R,0.0
3,/u/app,0.0
4,058.ba,0.0
5,1. FC Köln App,0.0
6,10 Best Foods for You,0.0
7,10 Minutes a Day Times Tables,0.0
8,10 WPM Amateur ham radio CW Morse code trainer,149.0
9,"10,000 Quotes DB (Premium)",495.0


In [164]:
df_ps.loc[df_ps['App'] == "10,000 Quotes DB (Premium)"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,InstallNumber,Income
8219,"10,000 Quotes DB (Premium)",BOOKS_AND_REFERENCE,4.1,70,3.5M,500+,Paid,0.99,Everyone,Books & Reference,"August 30, 2013",1.3,2.1,3500000.0,500,495.0


#### 12. For each app, compute its minimum and maximum Sentiment_polarity

In [55]:
df_ps_review.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [49]:
df_ps_review.groupby('App', as_index=False)[['Sentiment_Polarity']].min().head()

,App,Sentiment_Polarity
0,10 Best Foods for You,-0.8000
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.1125
2,11st,-1.0000
3,1800 Contacts - Lens Store,-0.3000
4,1LINE – One Line with One Touch,-0.8250


In [50]:
df_ps_review.groupby('App', as_index=False)[['Sentiment_Polarity']].max().head()

,App,Sentiment_Polarity
0,10 Best Foods for You,1.000000
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000
2,11st,1.000000
3,1800 Contacts - Lens Store,0.838542
4,1LINE – One Line with One Touch,1.000000
